In [1]:
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = 2000
epochs = 100
batch_size = 256
optimizer = "adam"
#learning_rate = 0.001
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo

features_prezzo = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "PSAR",
    "SUPERT", 
]

features_da_scalare_singolarmente = [
    "Volume",
    "ATR",
    "PSARaf",
    "ADX",
]

features_meno_piu = [
    "MACDh",    
    "AROONOSC",
    "TRIX",
    "TRIXs",
    "DM_OSC",
]

features_no_scala = [
    "SUPERTd",  
    "PSARr"
]

features_candele = [
    "CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]

elenco_targets = [
    "EMA_5",
    "EMA_20", 
    "EMA_50"    
]

col_features_prezzo = {col: idx for idx, col in enumerate(features_prezzo)}
col_features_da_scalare_singolarmente = {col: idx for idx, col in enumerate(features_da_scalare_singolarmente)}
col_features_meno_piu = {col: idx for idx, col in enumerate(features_meno_piu)}
col_features_no_scala = {col: idx for idx, col in enumerate(features_no_scala)}
col_features_candele = {col: idx for idx, col in enumerate(features_candele)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features_prezzo) + len(col_features_da_scalare_singolarmente) + len(col_features_meno_piu) + len(col_features_no_scala) + len(col_features_candele) 
n_targets = len(col_targets) 
    
def crea_modello(n_timesteps, giorni_previsione, n_targets, n_features):
        model = Sequential()

        # Layer LSTM iniziale
        model.add(LSTM(220, input_shape=(n_timesteps, n_features)))
        model.add(RepeatVector(giorni_previsione))

        # lstm-2
        model.add(LSTM(300, return_sequences=True,
                       kernel_regularizer=l2(4.0289e-06)))
        model.add(Dropout(1e-06))
        # lstm-3
#        model.add(LSTM(130, return_sequences=True,
#                       kernel_regularizer=l2(4.153520011432108e-05)))
#        model.add(Dropout(0.3))
        # lstm-4
#        model.add(LSTM(70, return_sequences=True,
#                       kernel_regularizer=l2(4.153520011432108e-05)))
#        model.add(Dropout(0.4))

        # Aggiunta di layer Dense
        model.add(TimeDistributed(Dense(120, activation='relu')))
        model.add(Dropout(0.4))

        model.add(TimeDistributed(Dense(n_targets, activation='linear')))

        model.compile(optimizer="adam",
                      loss='mean_absolute_percentage_error')

        return model

#from sklearn.preprocessing import PowerTransformer
#from sklearn.preprocessing import RobustScaler
#from sklearn.base import clone
#scaler = PowerTransformer()
#scaler = RobustScaler()



In [2]:
print("Importa librerie", end="", flush=True)

#from keras.losses import Huber
#print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
from sklearn.preprocessing import MinMaxScaler
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from tensorflow.keras.layers import Dropout
print(".", end="", flush=True)
from tensorflow.keras.layers import RepeatVector
print(".", end="", flush=True)
from tensorflow.keras.layers import TimeDistributed
print(".", end="", flush=True)
from keras.regularizers import l2
print(".", end="", flush=True)
from tensorflow.keras import Sequential
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, Dense
print(".", end="", flush=True)
#from tensorflow.keras.optimizers import Adam
#print(".", end="", flush=True)
#from sklearn.model_selection import train_test_split
#print(".", end="", flush=True)
from tensorflow.keras.models import load_model
print(".", end="", flush=True)
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)

print("Download lista ticker")
lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]

if os.path.exists('LSTM.keras'):
    print("Caricamento modello esistente")
    model = load_model("LSTM.keras")
else:
    print("Creazione modello")
    model = crea_modello(n_timesteps, giorni_previsione, n_targets, n_features)


Importa librerie..............=
Download lista ticker
Creazione modello


In [3]:
print("Download dati ticker validazione")
ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
ticker_val.index = ticker_val.index.date
print("Calcolo indicatori ticker validazione")
ticker_val = fx.crea_indicatori(ticker_val)
ticker_val.dropna(axis=0, inplace=True)

idx_val, X_val, Y_val, _ = fx.to_XY(ticker_val, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

for i_ticker in range (n_simboli_addestramento + 1):
    nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
    print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
    print("Download dati ticker")
    try:
        ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
        if ticker["Close"].iloc[-1] >= 1:
            ticker.index = ticker.index.date
            print("Calcolo indicatori ticker")
            ticker = fx.crea_indicatori(ticker)
            ticker.dropna(axis=0, inplace=True)

            print("Definizione features e target")
            idx, X, Y, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

            print(f"Addestramento modello epochs={epochs}, batch_size={batch_size}")
            model.fit(X, Y, epochs=epochs, batch_size=batch_size, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
            model.save("LSTM.keras")
    except Exception as e:
        print(e)
        continue


Download dati ticker validazione
Calcolo indicatori ticker validazione
1 di 2000: Ticker SRCL
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Addestramento modello epochs=100, batch_size=256
Epoch 1/100
13/13 [==============================] - 24s 1s/step - loss: 81.6387 - val_loss: 58.4083 - lr: 0.0010
Epoch 2/100
13/13 [==============================] - 17s 1s/step - loss: 65.4853 - val_loss: 59.6339 - lr: 0.0010
Epoch 3/100
13/13 [==============================] - 17s 1s/step - loss: 47.8323 - val_loss: 47.5394 - lr: 0.0010
Epoch 4/100
13/13 [==============================] - 17s 1s/step - loss: 48.2722 - val_loss: 44.9815 - lr: 0.0010
Epoch 5/100
13/13 [==============================] - 17s 1s/step - loss: 42.7471 - val_loss: 51.8243 - lr: 0.0010
Epoch 6/100
13/13 [==============================] - 17s 1s/step - loss: 38.1254 - val_loss: 45.2969 - lr: 0.0010
Epoch 7/100
13/13 [==============================] - 18s 1s/step - loss: 41.9002 - val_loss: 52.

13/13 [==============================] - 18s 1s/step - loss: 21.7038 - val_loss: 57.0821 - lr: 0.0010
5 di 2000: Ticker COLD
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Addestramento modello epochs=100, batch_size=256
Epoch 1/100
5/5 [==============================] - 10s 2s/step - loss: 31.7320 - val_loss: 35.4068 - lr: 0.0010
Epoch 2/100
5/5 [==============================] - 10s 2s/step - loss: 34.8408 - val_loss: 44.0782 - lr: 0.0010
Epoch 3/100
5/5 [==============================] - 10s 2s/step - loss: 35.2827 - val_loss: 40.9990 - lr: 0.0010
Epoch 4/100
5/5 [==============================] - 10s 2s/step - loss: 31.4461 - val_loss: 42.1862 - lr: 0.0010
Epoch 5/100
5/5 [==============================] - 10s 2s/step - loss: 28.3257 - val_loss: 38.2092 - lr: 0.0010
Epoch 6/100
5/5 [==============================] - 10s 2s/step - loss: 26.5401 - val_loss: 39.6830 - lr: 0.0010
Epoch 7/100
5/5 [==============================] - 10s 2s/step - loss: 25.454

Epoch 11/100
11/11 [==============================] - 15s 1s/step - loss: 24.6499 - val_loss: 43.7486 - lr: 0.0010
Epoch 12/100
11/11 [==============================] - 14s 1s/step - loss: 23.5375 - val_loss: 41.5600 - lr: 0.0010
Epoch 13/100
11/11 [==============================] - 15s 1s/step - loss: 22.6694 - val_loss: 43.9124 - lr: 0.0010
10 di 2000: Ticker ADI
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Addestramento modello epochs=100, batch_size=256
Epoch 1/100
13/13 [==============================] - 19s 1s/step - loss: 26.1768 - val_loss: 35.4969 - lr: 0.0010
Epoch 2/100
13/13 [==============================] - 19s 1s/step - loss: 24.5561 - val_loss: 36.1753 - lr: 0.0010
Epoch 3/100
13/13 [==============================] - 19s 1s/step - loss: 23.3862 - val_loss: 36.3006 - lr: 0.0010
Epoch 4/100
13/13 [==============================] - 18s 1s/step - loss: 22.7028 - val_loss: 36.0950 - lr: 0.0010
Epoch 5/100
13/13 [==============================]

8/8 [==============================] - 14s 2s/step - loss: 19.3955 - val_loss: 55.0973 - lr: 0.0010
Epoch 32/100
8/8 [==============================] - 14s 2s/step - loss: 19.0485 - val_loss: 54.5919 - lr: 0.0010
14 di 2000: Ticker SUPN
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Addestramento modello epochs=100, batch_size=256
Epoch 1/100
11/11 [==============================] - 19s 2s/step - loss: 49.0947 - val_loss: 60.4746 - lr: 0.0010
Epoch 2/100
11/11 [==============================] - 19s 2s/step - loss: 49.2642 - val_loss: 47.4124 - lr: 0.0010
Epoch 3/100
11/11 [==============================] - 18s 2s/step - loss: 45.0888 - val_loss: 37.2387 - lr: 0.0010
Epoch 4/100
11/11 [==============================] - 18s 2s/step - loss: 33.8883 - val_loss: 36.5956 - lr: 0.0010
Epoch 5/100
11/11 [==============================] - 18s 2s/step - loss: 35.2497 - val_loss: 37.4778 - lr: 0.0010
Epoch 6/100
11/11 [==============================] - 17s 2s/step - 

10/10 [==============================] - 14s 1s/step - loss: 27.2346 - val_loss: 41.4462 - lr: 0.0010
Epoch 13/100
10/10 [==============================] - 15s 2s/step - loss: 26.1247 - val_loss: 41.8165 - lr: 0.0010
Epoch 14/100
10/10 [==============================] - 14s 1s/step - loss: 25.9974 - val_loss: 42.2330 - lr: 0.0010
Epoch 15/100
10/10 [==============================] - 14s 1s/step - loss: 25.6262 - val_loss: 42.9690 - lr: 0.0010
Epoch 16/100
10/10 [==============================] - 19s 2s/step - loss: 25.2188 - val_loss: 42.4428 - lr: 0.0010
Epoch 17/100
 8/10 [=======================>......] - ETA: 2s - loss: 24.8525


KeyboardInterrupt



In [8]:

print("Download dati ticker per previsione")
dati_previsione = yf.download(simbolo_test, start='2010-01-01', end='2023-12-31', progress=False)
dati_previsione.index = dati_previsione.index.date
dati_previsione = fx.crea_indicatori(dati_previsione)
dati_previsione.iloc[:100] = dati_previsione.iloc[:100].dropna(axis=0)

indice_prev, X_prev, Y_prev, scaler_prezzo = fx.to_XY(dati_previsione, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=False)

pred = model.predict(X_prev)
#pred_loss, pred_mae = model.evaluate(X_prev, Y_prev)
X_prev = X_prev.reshape(-1, n_timesteps * n_features)
X_prev = scaler_prezzo.inverse_transform(X_prev)
X_prev = X_prev.reshape(-1, n_timesteps, n_features)
pred = pred.reshape(-1, giorni_previsione * n_targets)
pred = scaler_prezzo.inverse_transform(pred)
pred = pred.reshape(-1, giorni_previsione, n_targets)

col_analisi = "EMA_5"

import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.subplots as sp
import yfinance as yf

def grafico1(dati_previsione, pred):
    dati_previsione = dati_previsione.loc[indice_prev]
    new_dates = pd.bdate_range(start=dati_previsione.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates.date)
    dati_previsione = pd.concat([dati_previsione, df_new])
    risultato = pd.DataFrame({"Previsione": pred[:, giorni_previsione-1, col_targets[col_analisi]].round(2)}, index=dati_previsione.index[giorni_previsione:])
    risultato = pd.concat([dati_previsione[col_analisi], risultato], axis=1)
    
    target = go.Scatter(
        x = risultato.index,
        y = risultato[col_analisi],
        mode = 'lines',
        line = dict(color='rgba(0, 0, 0, .9)'),
        name = col_analisi
    )

    previsione = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'],
        mode = 'lines',
        line = dict(color='rgba(0, 0, 250, .9)'),
        name = 'Previsione'
    )
    '''
    err_meno = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'] - err, 
        mode = 'lines',
        line = dict(
            color='rgba(0, 0, 250, .9)',
            width=1
        ),
        connectgaps = False,
        name = 'err-'
    )                
    err_piu = go.Scatter(
        x = risultato.index,
        y = risultato['Previsione'] + err, 
        mode = 'lines',
        line = dict(
            color='rgba(0, 0, 250, .9)',
            width=1
        ),
        fill='tonexty',
        fillcolor='rgba(0, 0, 250, .2)', # Puoi cambiare il valore di alpha per regolare la trasparenza
        connectgaps = False,
        name = 'err+'
    )
    '''
    layout = dict(xaxis = dict(autorange=True),
                  yaxis = dict(title = 'Previsioni', autorange=True),
                  autosize = True,
                  margin = go.layout.Margin(
                      l=0,  # Sinistra
                      r=0,  # Destra
                      b=0,  # Basso
                      t=50,  # Alto
                      pad=0  # Padding
                  ),
                  legend = dict(traceorder = 'normal', bordercolor = 'black')
    )
    fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.update_layout(layout)
    fig.add_trace(target, row=1, col=1)
    fig.add_trace(previsione, row=1, col=1)
    #fig.add_trace(err_meno, row=1, col=1)
    #fig.add_trace(err_piu, row=1, col=1)
    pyo.plot(fig, filename="grafico1.html", auto_open=True)




Download dati ticker per previsione
107/107 [==============================] - 6s 55ms/step


In [14]:
def grafico2(dati_previsione, pred):
    dati2 = dati_previsione.dropna(axis=0).iloc[-60:]
    new_dates = pd.bdate_range(start=dati2.index[-1] + pd.Timedelta(days=1), periods=giorni_previsione)
    df_new = pd.DataFrame(index=new_dates.date)

    val_prev2 = (pred[-1, i, col_targets[col_analisi]] for i in range(giorni_previsione))
    prev2 = pd.DataFrame(val_prev2, columns=["Previsione"], index=df_new.index)
    riga_iniziale = pd.DataFrame({"Previsione": [dati2[col_analisi].iloc[-1]]}, index=[dati2.index[-1]])
    prev2 = pd.concat([riga_iniziale, prev2], axis=0)
    
    target = go.Scatter(
        x = dati2.index,
        y = dati2[col_analisi].round(2),
        mode = 'lines',
        line = dict(color='rgba(0, 0, 0, .9)'),
        name = col_analisi
    )

    previsione = go.Scatter(
        x = prev2.index,
        y = prev2['Previsione'].round(2),
        mode = 'lines',
        line = dict(color='rgba(250, 128, 0, .9)'),
        name = 'Previsione'
    )
    layout = dict(xaxis = dict(autorange=True),
                  yaxis = dict(title = 'Previsioni', autorange=True),
                  autosize = True,
                  margin = go.layout.Margin(
                      l=0,  # Sinistra
                      r=0,  # Destra
                      b=0,  # Basso
                      t=50,  # Alto
                      pad=0  # Padding
                  ),
                  legend = dict(traceorder = 'normal', bordercolor = 'black')
    )
    fig = sp.make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.update_layout(layout)
    fig.add_trace(target, row=1, col=1)
    fig.add_trace(previsione, row=1, col=1)
    pyo.plot(fig, filename="grafico2.html", auto_open=True)



In [17]:
#grafico1(dati_previsione, pred)

In [16]:
grafico2(dati_previsione, pred)

In [ ]:
df_risultato = pd.DataFrame()